## 1. Hàm hỗ trợ

In [ ]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

## 2. Hàm nhúng

In [ ]:
def embed(msg_file, cover_text_file, text_width, stego_text_file):
    '''
    Nhúng tin mật vào văn bản bằng phương pháp chèn khoảng trắng 
    để văn bản được căn lề 2 bên.
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_text_file (str): Tên file chứa cover text.
        text_width (int): Chiều dài của dòng sau khi được căn lề.
        stego_text_file (str): Tên file chứa stego text.
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    '''
    theo qui ước: bit 0 = 1 khoảng trắng chèn thêm 1 khoảng trắng + 1
    khoảng trắng, bit 1 = 1 khoảng trắng + 1 khoảng trắng chèn thêm 1
    khoảng trắng; b += 1
    • Ngược lại: lần đầu (trong cả quá trình nhúng) nhúng bit 1, những lần
    sau nhúng bit 0
    
    • Với các khoảng trắng còn lại của dòng l: nếu min(m, n-m) = n-m thì chèn
    thêm một khoảng trắng vào mỗi khoảng trắng
    
    ▪ Còn nếu m ≥ n: không nhúng được bit, nhưng mỗi khoảng trắng cần được
    chèn thêm ít nhất một khoảng trắng để dòng l được căn lề
    
     l += 1
     Nếu vẫn chưa nhúng được bit 1 (trong đuôi 100...): NHÚNG
    THẤT BẠI!
    '''
    with open(cover_text_file, 'r') as f:
      cover_text_lines = f.read().splitlines()
    
    with open(msg_file, 'r') as f:
        msg = f.read()
    
    # msg_bits = bitarray.bitarray()
    # msg_bits.fromstring(msg)
    msg_bits = tobits(msg)
    stego_text = ''
    b = 0 # Chỉ số duyệt từng bit của msg_bits
    l = 0 # Chỉ số duyệt từng dòng của cover_text
    last_bit = False
    for line in cover_text_lines:
        if l < len(cover_text_lines)-1: # Có thể nhúng được bit
            if cover_text_lines[l+1] != '':
                line_width = len(line)
                if line_width < text_width:
                    n = line.count(' ') # n = số khoảng trắng của dòng l
                    m = text_width - line_width # m = số khoảng trắng cần phải chèn để dòng l được căn lề = text_width – len(dòng l)
                    if 0 < m < n: # Nhúng được min(m, n-m) bit
                        num_bits = min(m, n-m) # cặp khoảng trắng đầu của dòng l
                        words = line.split()
                        for i in range(num_bits):
                            if b < len(msg_bits):
                                if msg_bits[b] == 0:
                                    words.insert(i*3+1, ' ')
                                else:
                                    words.insert(i*3+2, ' ')
                                b += 1
                            else:
                                if last_bit == False:
                                    words.insert(i*3+2, ' ')
                                    last_bit = True
                                else:
                                    words.insert(i*3+1, ' ')
                        
                        if num_bits == n-m:
                            j = m-num_bits
                            while j > 0:
                                words.insert(len(words)-j, ' ')
                                j -= 1
                                
                        for w in range(len(words)):
                            stego_text += words[w]
                            if words[w] != ' ':
                                if w < len(words)-1:
                                    stego_text += ' '
                    else:
                        needed_spaces = m
                        text_list = list(line)
                        for i in range(int(-(-(m-n) // 1))+1):
                            for j, c in enumerate(line):
                                if needed_spaces > 0:
                                    if c == ' ':
                                        text_list[j] = ' '+' '*(i+1)
                                        needed_spaces -= 1
                        stego_text += ''.join(text_list)
                else:
                    stego_text += line
            else:
                stego_text += line
            stego_text += '\n'
        else:
            stego_text += line
        l += 1
    #print(stego_text)
    with open(stego_text_file, 'w') as f:
        f.write(stego_text)
        
    return last_bit

In [ ]:
# TEST 1            
result = embed('msg.txt', 'cover.txt', 70, 'stego.txt')
assert result == True
with open('stego.txt', 'r') as f:
    stego_text = f.read()
with open('correct_stego.txt', 'r') as f:
    correct_stego_text = f.read()
assert stego_text == correct_stego_text

In [ ]:
# TEST 2
result = embed('msg2.txt', 'cover.txt', 70, 'stego.txt')
assert result == False




## 3. Hàm rút trích

In [ ]:
def extract(stego_text_file, extr_msg_file):
    '''
    Hàm rút trích tin mật đã được nhúng bằng phương pháp chèn khoảng trắng.
    
    Các tham số:
        stego_text_file (str): Tên file chứa stego text.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''
    with open(stego_text_file, 'r') as f:
        stego_text_lines = f.read().splitlines()
    
    extr_msg_bits = []
    for line in stego_text_lines:
        words = line.split(' ')
        if len(words) > 2:
            space_list = []
            for i in range(len(words)-1):
                if words[i] != '':
                    if words[i+1] == '':
                        space_list.append('  ')
                    else:
                        space_list.append(' ')
            
            c_spaces = [space_list[x:x+2] for x in range(0, len(space_list), 2)]
            for p in c_spaces:
                if len(p) == 2:
                    if p[0] == '  ' and p[1] == ' ': #Nếu gặp 2 khoảng trắng + 1 khoảng trắng: thêm bit 0 vào extracted_msg_bits
                        extr_msg_bits.append(0)
                    elif p[0] == ' ' and p[1] == '  ': #nếu gặp 1 khoảng trắng + 2 khoảng trắng: thêm bit 1 vào extracted_msg_bits
                        extr_msg_bits.append(1)

    last_index = len(extr_msg_bits)-1
    for i in reversed(range(len(extr_msg_bits))): #Cắt đoạn đuôi 100.. ra khỏi extracted_msg_bits
        if extr_msg_bits[i] == 1:
            last_index = i
            break
    extr_msg = frombits(extr_msg_bits[:last_index]) #lấy dữ liệu trừ đoạn đã cắt.
    with open(extr_msg_file, 'w') as f:
        f.write(extr_msg)

In [ ]:
# TEST
extract('correct_stego.txt', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('correct_extr_msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg